<a href="https://colab.research.google.com/github/antiantono/bts_assignments/blob/main/advanced_analysis/MBA/MBA_Anti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment : Market Basket Analysis
### Subject : Advanced Data Analysis
### Submitted by : Anti Antono

# 1 Data Preparation

In [1]:
# Libraries 
import numpy as np
import pandas as pd

# Multiple output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 1.1 Read dataset

Dataset includes two files:
* **transactions** : baskets / lists containing purchased items
* **items** : list of item codes and prices

Both files were downloaded from KNIME using the csv writer node.

![](https://drive.google.com/uc?export=view&id=11I9ywjkTSx8IAF-9D6HsIHPotMPIa59u)


In [2]:
# Read csv files
PATH = 'https://raw.githubusercontent.com/antiantono/bts_assignments/main/advanced_analysis/MBA/'

items = pd.read_csv(PATH + 'items.csv')
baskets = pd.read_csv(PATH + 'transactions.csv', header=0)

In [3]:
# Inspect file content
print("# of items: {}, # of baskets: {}".format(len(items), len(baskets)))
print("items\n", items.head(3))
print("baskets\n", baskets.head(3))

# of items: 246, # of baskets: 2869
items
    Item      Price   ProductName
0     0   2.511933  swiss cheese
1     1  52.441562   Cherry coke
2     2  28.215382      Bio Coke
baskets
                                                 Col0
0  224 80 109 177 50 43 83 173 70 202 94 227 162 ...
1  56 95 106 186 103 170 69 198 186 211 83 24 78 ...
2  9 196 184 119 88 196 222 94 212 187 95 3 224 5...


## 1.2 Reformat dataset

Process:
* Parse the original baskets into a list of all items included the basket 
* Extract all baskets from dataframe into a list
* Convert into one-hot matrix of transactions x items using Mlxtend encoder 

In [4]:
# Rename column + split items in the baskets
baskets.columns = ['original']
baskets['split'] = baskets['original'].apply(lambda basket: str(basket).split())
baskets.head()

,original,split
0,224 80 109 177 50 43 83 173 70 202 94 227 162 ...,"[224, 80, 109, 177, 50, 43, 83, 173, 70, 202, ..."
1,56 95 106 186 103 170 69 198 186 211 83 24 78 ...,"[56, 95, 106, 186, 103, 170, 69, 198, 186, 211..."
2,9 196 184 119 88 196 222 94 212 187 95 3 224 5...,"[9, 196, 184, 119, 88, 196, 222, 94, 212, 187,..."
3,228 9 193 127 163 117 24 34 204 163 48 74 69 2...,"[228, 9, 193, 127, 163, 117, 24, 34, 204, 163,..."
4,94 9 22 133 107 228 77 173 38 109 32 31 110 79...,"[94, 9, 22, 133, 107, 228, 77, 173, 38, 109, 3..."


In [5]:
# Extract all baskets into a giant list
baskets_list = []
for i in range(0, baskets.shape[0]):
    baskets_list.append(baskets.split.values[i])

In [6]:
# Encode into one-hot matrix 
from mlxtend.preprocessing import TransactionEncoder
encoder = TransactionEncoder()
baskets_onehot = encoder.fit(baskets_list).transform(baskets_list)

# Put into dataframe
baskets_df = pd.DataFrame(baskets_onehot, columns= encoder.columns_)
baskets_df.shape
baskets_df.head(3)

(2869, 243)

,0,1,10,100,101,102,103,104,105,106,107,108,109,11,110,111,112,113,114,115,116,117,118,119,12,120,121,122,123,124,125,126,127,128,129,13,130,131,132,133,...,62,63,64,65,66,67,68,69,7,70,71,72,73,74,75,76,77,78,79,8,80,81,82,83,84,85,86,87,88,9,90,91,92,93,94,95,96,97,98,99
0,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,True,False,False,True,False,True,False,False,False,False,False
1,False,False,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,True,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,True,True,True,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,False,True,True,False,False,False,False


In [7]:
# Some sense check on the basket size - looks ok
print("Smallest basket size:", baskets_df.sum(axis=1).min())
print("Average basket size:", baskets_df.sum(axis=1).mean())
print("Largest basket size:", baskets_df.sum(axis=1).max())

Smallest basket size: 1
Average basket size: 15.849773440223075
Largest basket size: 51


## 1.3 Adding item names

Approach : 
* Transpose baskets_df into a matrix of item x transactions
* Join with item dataset on key = Item (item code)
* Reset the index to ProductName
* Clean up extra columns and tranpose back, now with ProductName as column names

In [8]:
# Transpose the basket, so Item becomes a column
baskets_dft = baskets_df.transpose().reset_index().astype(int)
baskets_dft.rename(columns = {'index': 'Item'}, inplace= True)
baskets_dft.head(3)

,Item,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,2829,2830,2831,2832,2833,2834,2835,2836,2837,2838,2839,2840,2841,2842,2843,2844,2845,2846,2847,2848,2849,2850,2851,2852,2853,2854,2855,2856,2857,2858,2859,2860,2861,2862,2863,2864,2865,2866,2867,2868
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
1,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
# Join with list of items table and use ProductName as index
baskets_dft_joined = pd.merge(baskets_dft, items, how= 'left', on= 'Item')
baskets_dft_joined = baskets_dft_joined.set_index('ProductName').drop(columns= ['Item', 'Price'])
baskets_dft_joined.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,2829,2830,2831,2832,2833,2834,2835,2836,2837,2838,2839,2840,2841,2842,2843,2844,2845,2846,2847,2848,2849,2850,2851,2852,2853,2854,2855,2856,2857,2858,2859,2860,2861,2862,2863,2864,2865,2866,2867,2868
ProductName,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
swiss cheese,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
Cherry coke,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
cheddar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
omlett,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Parsnips,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [10]:
# Transpose back into final dataset
baskets_final = baskets_dft_joined.sort_index().transpose()
print("Total number of baskets:", len(baskets_final))
baskets_final.head(3)

Total number of baskets: 2869


ProductName,American cheese,Apple Juice,Apple cake,Apricot,Artichoke,Asparagus,Aubergene,Avocado,Beans,Beet,Best white tea,Bio Coke,Bio lemonade,Blackcurrant,Blueberry,Bordeaux (Red Wine),Breadfruit,Brezels,Broccoli,Brussels sprouts,Cabbage,Carrot,Cauliflower,Celeriac,Celery,Champagne Esprit du Siècle,Chard,Chardonnay(White wine),Cherimoya,Cherry coke,Chicory,Chinese Food,Choclate cake,Clementine,Coconut,Coke,Colby cheese,Collards,Corn,Cranberry,...,quail eggs,rasberries,red herb,red patches,romano,safran,salmone,sauerkraut,scrambled egg,shrimps,shrimps,snails,sour cream,spare ribs,sparkling wine,spiny lobster,spritzer,strawberries,sunflower seed bread,swiss cheese,swordfish,tap water,tofu,tomato,trout,truffles,turkey,unleavened bread,uso,water,water with gas,white kohl,white truffles,whole-grain bread,wodka,yoghurt,yoghurt lowfat,yoghurt with fruit,yoghurt with fruit lowfat,zuccini
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,1,0,0,1,0,1,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0


# 2 Generate item sets and association rules

Some decisions and rationale in generating frequent sets:
* The frequent sets are limited to **two items** in a set for simplicity
* A fairly low threshold of **support (minimum 0.5%)** was applied, in order to obtain a wide range of rules that can include sets with low incidence items. Initial trials with higher threshold resulted in sets too much dominated by high support items.  

Criteria in generating association rules, and rationale: 
* Lift was used as the key metric, as it's intuitive to interpret and it ensures the strongest association rules are significant i.e. they did not arise simply due to chance (esp. for high incidence items)
* Lift > 1 means the association rule is stronger than by chance, but a higher threshold (1.25) was applied to increase the strength of the association

In [11]:
# Libraries
from mlxtend.frequent_patterns import apriori, association_rules 

In [12]:
# Check the lowest support for individual items
baskets_final.mean().sort_values().head()

ProductName
Cabbage      0.015336
Pumpkins     0.015336
Fig          0.016731
Artichoke    0.017079
Persimmon    0.017079
dtype: float64

In [13]:
# GENERATING FREQUENT SETS
# We want a list with a good range of options
# A couple of limits set : support > .5% of total baskets, and max length of items in set = 2

frequent_sets = apriori(baskets_final, min_support= 0.005, max_len= 2, use_colnames=True)
print("# of frequent sets in our list:", len(frequent_sets))

# of frequent sets in our list: 10530


In [14]:
# GENERATING ASSOCIATION RULES
# Create association rules based on the frequent sets we created

rules = association_rules(frequent_sets, metric= 'lift', min_threshold= 1.25)
print("# of rules:", len(rules))
rules.head()

# of rules: 13728


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(American cheese),(Bio Coke),0.046358,0.140467,0.010805,0.233083,1.659341,0.004293,1.120764
1,(Bio Coke),(American cheese),0.140467,0.046358,0.010805,0.076923,1.659341,0.004293,1.033113
2,(Bio lemonade),(American cheese),0.087138,0.046358,0.006971,0.080000,1.725714,0.002932,1.036568
3,(American cheese),(Bio lemonade),0.046358,0.087138,0.006971,0.150376,1.725714,0.002932,1.074430
4,(American cheese),(Brezels),0.046358,0.092367,0.007320,0.157895,1.709434,0.003038,1.077815


# 3 Suggested items

In [32]:
# RECOMMENDATION FUNCTION
# Suggests other items to buy based on the rules we built

def what_else(item, max_reco=5, rules=rules):
  ''' 
  This function will recommend up to 5 other items that you may be interested in buying, based on what you're buying today.
  '''

  # Pick relevant rules and sort according to lift values
  rules_sorted = rules[rules['antecedents'] == {item}].sort_values('lift', ascending= False)

  # Get the recommended items
  reco_items = rules_sorted['consequents'].apply(lambda x: ', '.join(list(x))).astype("str")
  
  # Limit to required max. number of recommendations
  print(f"If you are buying {item} today, you may also be interested in:")
  for i in range(0, max_reco):
    print("- ", reco_items.values[i])

In [16]:
# RANDOM ITEM SELECTOR
# Because you may not know what you want to buy today

def inspire_me():
  index = np.random.randint(0, len(items))
  item = items.ProductName.iloc[index]
  print(f"How about ... {item}?\n")
  return item

In [17]:
# Let's add some other things into your shopping basket!
# If you are buying Tuna ...

what_else('Tuna')

If you are buying Tuna today, you may also be interested in:
-  shrimps
-  Broccoli
-  mango
-  Leek
-  Beet


In [34]:
# If you don't know what to buy, pick a random item using the inspire_me() function
# And see what else the recommender function suggests

what_else(inspire_me())

How about ... beer?

If you are buying beer today, you may also be interested in:
-  shrimps
-  Brezels
-  pork
-  Sharon Fruit (Persimmon)
-  pita bread
